In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, LassoCV
import statsmodels.api as sm

In [4]:
import preprocessing as preproc
# help(preproc)

In [5]:
int_freq = 30

# Data Collection

In [10]:
start_date = '2009-01-01'
end_date = '2024-12-31'
date_range = preproc.create_date_range(start_date, end_date)

### Downstream Data (Inflow)

In [13]:
inflow_site = '14128600'

## Collect Dataset #1
df1 = preproc.preprocess(inflow_site, int_freq, start_date, end_date)

COLUMBIA RIVER AT STEVENSON, WA
Removed 24 duplicate rows

Removed 0 rows with NaN in '00065'

Imputed 201 rows


### Upstream Data (Outflow)

In [15]:
outflow_site = '14105700'

## Collect Dataset #2
df2 = preproc.preprocess(outflow_site, int_freq, start_date, end_date)

COLUMBIA RIVER AT THE DALLES, OR
Removed 32 duplicate rows

Removed 437 rows with NaN in '00065'

Imputed 887 rows


### Data Export

In [78]:
df2.rename(columns={'00065': 'up_outflow'}, inplace=True)

In [74]:
df1.rename(columns={'00065': 'down_inflow'}, inplace=True)

In [76]:
df1.head()

,site_no,down_inflow
datetime,,
2009-01-01 00:00:00,14128600,76.11
2009-01-01 00:30:00,14128600,76.19
2009-01-01 01:00:00,14128600,76.19
2009-01-01 01:30:00,14128600,76.25
2009-01-01 02:00:00,14128600,76.20


In [80]:
df2.head()

,site_no,up_outflow
datetime,,
2009-01-01 00:00:00,14105700,76.61
2009-01-01 00:30:00,14105700,76.56
2009-01-01 01:00:00,14105700,76.47
2009-01-01 01:30:00,14105700,76.27
2009-01-01 02:00:00,14105700,76.39


# Training & Test Set

In [17]:
train_start_year = '2008'
test_start_year = '2024'

In [18]:
## Merge downstream and upstream data

df_range = preproc.dataset_merge(df1, df2, train_start_year, "full")

In [19]:
## Create lag features

p = 7 
up_feat = True
down_feat = True 

df_reg, feature_cols = preproc.create_lag_features(df_range, p, up_feat, down_feat)

In [20]:
## Split the data into training and testing sets

train_data = df_reg[(df_reg.index < test_start_year)]
test_data = df_reg[(df_reg.index >= test_start_year)]

X_train = train_data[feature_cols]
y_train = train_data['y_norm']

X_test = test_data[feature_cols]
y_test = test_data['y_norm']

# M0: Previous Term Forecaster

In [22]:
## Simulate Model Forecasting
# Model Output (y_hat): y[t-1]
# Ground Truth (y): y[t]

fitted_vals0 = y_train[:-1].values
y_train0 = y_train[1:].values

## Residuals

In [51]:
## Residuals = y[t] - y[t-1]

residuals0 = y_train0 - fitted_vals0

In [ ]:
plt.hist(residuals0, bins = 100)
plt.title('M0: Training Residuals')
plt.xlim([-.5, .5])
plt.show()

In [ ]:
preproc.plot_variance("M0", fitted_vals0, residuals0)

## Evaluation

In [ ]:
y_pred0 = y_test[:-1] # Exclude the last element
y_test0 = y_test[1:]  # Exclude the first element

In [ ]:
preproc.print_test_stats(y_test0, y_pred0)

In [ ]:
preproc.plot_forecasts("M0", y_test0, y_pred0)

# M1: Linear Regression

In [ ]:
# Step 1: Add constant (intercept)
X_train_const = sm.add_constant(X_train)

# Step 2: Fit OLS model
model = sm.OLS(y_train, X_train_const).fit(cov_type='HC1') # Heteroskedasticity-consistent

# Step 3: View summary
print(model.summary())

In [ ]:
# Extract coefficients and p-values into a tidy DataFrame
coef_df = pd.DataFrame({
    'feature': model.params.index,
    'coefficient': model.params.values
    #'p_value': model.pvalues.values,
    #'t_value': model.tvalues.values
})

# Optional: sort by absolute coefficient size
coef_df = coef_df.reindex(coef_df['coefficient'].abs().sort_values(ascending=False).index)

print(coef_df)

## Residuals

In [ ]:
residuals1 = model.resid
fitted_vals1 = model.fittedvalues

In [ ]:
# Checking for Heteroskedasticity

preproc.plot_variance("M1", fitted_vals1, residuals1)

## Evaluation

In [ ]:
X_test_const = sm.add_constant(X_test)  # Add intercept term
X_test_const['const'] = np.ones(len(X_test))
X_test_const = pd.DataFrame(X_test_const, columns=model.model.exog_names)
y_pred1 = model.predict(X_test_const)

In [ ]:
preproc.plot_forecasts("M1", y_test, y_pred1)

In [ ]:
preproc.print_test_stats(y_test, y_pred1)

# M2: LASSO

In [ ]:
spline_feat = False

if spline_feat:
    lasso_train = X_train_M3_full.copy()
    lasso_test = X_test_M3_full.copy()
else:
    lasso_train = X_train.copy() 
    lasso_test = X_test.copy()

In [ ]:
## Cross-Validation for Hyperparamter Finetuning

lasso_cv = LassoCV(cv=5, positive=True).fit(lasso_train, y_train)
alpha_star = lasso_cv.alpha_
print("Best alpha:", alpha_star)

In [ ]:
## Create and fit LASSO model

#lasso = Lasso(alpha=0, positive=True)  # alpha is the regularization strength (higher = more shrinkage)
lasso = Lasso(alpha=alpha_star, positive=True)
lasso.fit(lasso_train, y_train)

In [ ]:
## Make predictions

y_pred2 = pd.Series(lasso.predict(lasso_test))

In [ ]:
## View coefficients

coef_df = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': lasso.coef_
})

# Optional: Sort by magnitude or filter out zeros
coef_df = coef_df[coef_df['coefficient'] != 0]  # remove zeroed-out features
coef_df = coef_df.sort_values(by='coefficient', key=abs, ascending=False)

print(coef_df)

## Residuals

In [ ]:
fitted_vals2 = lasso.predict(lasso_train)

In [ ]:
residuals2 = y_train - fitted_vals2

In [ ]:
plt.hist(residuals2, bins = 100)
plt.title('M2: Training Residuals')
plt.xlim([-.5, .5])
plt.show()

In [ ]:
# Checking for Heteroskedasticity

preproc.plot_variance("M2", fitted_vals2, residuals2)

## Evaluation

In [ ]:
preproc.plot_forecasts("M2", y_test, y_pred2)

In [ ]:
preproc.print_test_stats(y_test, y_pred2)

# M3: Spline

In [ ]:
from patsy import dmatrix

In [ ]:
## Create numerical & continuous time variable

X_train_M3 = X_train_const.copy()
X_train_M3["t"] = (X_train_M3.index - X_train_M3.index[0]).total_seconds() / (3600 * 24)

In [ ]:
## Generate spline

# Natural cubic spline with 4 degrees of freedom (adjust as needed)
spline = dmatrix("bs(t, df=4, include_intercept=False)", data=X_train_M3, return_type='dataframe')

In [ ]:
## Combine spline basis with lag features

X_train_M3_full = pd.concat([X_train_M3.drop(columns="t"), spline], axis=1)

In [ ]:
## Fit the model 

model3 = sm.OLS(y_train, X_train_M3_full).fit()
print(model3.summary())

## Residuals

In [ ]:
residuals3 = model3.resid
fitted_vals3 = model3.fittedvalues

In [ ]:
preproc.plot_variance("M3", fitted_vals3, residuals3)

## Evaluation

In [ ]:
## Prepare test dataset

X_test_M3 = X_test_const.copy()
X_test_M3["t"] = (X_test.index - X_train.index[0]).total_seconds() / (3600 * 24)

spline_test = dmatrix("bs(t, df=4, include_intercept=False)", data=X_test_M3, return_type='dataframe')
X_test_M3_full = pd.concat([X_test_M3.drop(columns="t"), spline_test], axis=1)

In [ ]:
y_pred3 = model3.predict(X_test_M3_full)

In [ ]:
preproc.plot_forecasts("M3", y_test, y_pred3)

In [ ]:
preproc.print_test_stats(y_test, y_pred3)

# M4: Prophet

In [ ]:
prophet_df = y_train.to_frame(name='y')
prophet_df = prophet_df.reset_index()  
prophet_df.columns = ['ds', 'y']    

In [ ]:
from prophet import Prophet

m = Prophet()
m.fit(prophet_df)

In [ ]:
future = y_test.index.to_frame(name='ds')
future = future.reset_index()  
future = future.drop('datetime', axis = 1)

In [ ]:
full_future = pd.concat([prophet_df['ds'], future])

In [ ]:
forecast = m.predict(full_future)

# forecast.columns

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

## Residuals

In [ ]:
# Merge actuals and predictions
prophet_merged = prophet_df[['ds', 'y']].merge(forecast[['ds', 'yhat']], on='ds')
prophet_merged['residual'] = prophet_merged['y'] - prophet_merged['yhat']

In [ ]:
prophet_merged.head()

In [ ]:
prophet_merged_train = prophet_merged.iloc[:len(y_train)]

In [ ]:
plt.hist(prophet_merged_train.residual, bins = 40)
plt.title('Training Residuals')
plt.show()

In [ ]:
# Checking for Heteroskedasticity

preproc.plot_variance("M4", prophet_merged_train.yhat, prophet_merged_train.residual, bounds=False)

## Evaluation

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)


In [ ]:
y_pred4 = forecast.yhat[-len(y_test):]

In [ ]:
preproc.plot_forecasts("M4", y_test, y_pred4)

In [ ]:
preproc.print_test_stats(y_test, y_pred4)